In [1]:
import numpy as np
import ctqmc
import importlib
import conf
import ctqmc
import mpl_interface
import collections
import line_profiler
from mpl_interface import plt, spinplot, bondplot, cutplot
importlib.reload(conf)
importlib.reload(ctqmc)
importlib.reload(mpl_interface)

<module 'mpl_interface' from '/root/study/ctqmc/mpl_interface.py'>

In [2]:
pr = line_profiler.LineProfiler()

In [16]:
p = {'beta': 100, 'J': 1.0, 'Hx': 1.0,'L': 100}
u = ctqmc.Updater(**p)
pr.add_function(u.update)
pr.add_function(u.cuttaux.add_cut)

In [17]:
%time u.remove_cut()
%time u.add_cut()
%time u.update_bond()
%time u.update()

CPU times: user 175 µs, sys: 80 µs, total: 255 µs
Wall time: 257 µs
CPU times: user 10 ms, sys: 315 µs, total: 10.3 ms
Wall time: 8.46 ms
CPU times: user 5.33 ms, sys: 68 µs, total: 5.4 ms
Wall time: 5.25 ms
CPU times: user 9.52 ms, sys: 0 ns, total: 9.52 ms
Wall time: 9.51 ms


In [18]:
pr.run('u.update()')
pr.run('u.cuttaux.add_cut()')

In [19]:
pr.print_stats()

Timer unit: 1e-09 s

Total time: 0.0162451 s
File: /root/study/ctqmc/conf.py
Function: add_cut at line 117

Line #      Hits         Time  Per Hit   % Time  Line Contents
   117                                               def add_cut(self, ):
   118       101      12075.0    119.6      0.1          for ct in self.cuttaux:
   119       100   16233002.0 162330.0     99.9              ct.add_cut()

Total time: 0.016845 s
File: /root/study/ctqmc/ctqmc.py
Function: add_cut at line 76

Line #      Hits         Time  Per Hit   % Time  Line Contents
    76                                               def add_cut(self):
    77         1   16845004.0    2e+07    100.0          self.cuttaux.add_cut()

Total time: 0.029729 s
File: /root/study/ctqmc/ctqmc.py
Function: update at line 82

Line #      Hits         Time  Per Hit   % Time  Line Contents
    82                                               def update(self):
    83         1    1655572.0    2e+06      5.6          self.cluster = self._